# [Not Working] StatsModels Kalman filter
> This is the implementation of kalman filters using statsmodels statespace models, however it is not working for imputation because it doesn't support missing data (it just removes the missing observation from the dataset ...)

#| hide
#| default_exp kalman.imputation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from fastcore.basics import store_attr
import statsmodels.api as sm
import numpy as np
from meteo_imp.results import ImputationResult

to work properly with the statsmodel the dataframe can have missing data, but the data should be have the index properly set

In [ ]:
class StateSpaceImputation:
    """Imputation using a statsmodel state space model"""
    def __init__(self, data: pd.DataFrame,
                 model: sm.tsa.statespace.MLEModel, # a subclass of MLEModel tto be used as model
                 model_args: dict = {}, # Optional args for model
                 pred_all: bool = False, # If the dataset should be replaced by the model predictions
                 # or only the gaps imputed using the model
                 int_index = True):
        store_attr(but='model')
        if int_index: self.data.set_index(np.arange(0, len(self.data)))
        
        self.train_idx = ~self.data.isna().any(axis=1)
        self.train_data = self.data[self.train_idx]
        self.model = model(self.train_data.to_numpy(), **model_args)
    def fit(self) -> 'StateSpaceImputation':
        """Fit model parameters"""
        self.result = self.model.fit()
        return self

    def impute(self):
        """Impute data in tidy format using model"""
        # prediction for all dataset
        pred = self.result.get_prediction(information_set="smooth")


    # this is for integration with the rest of meteo_imp
    def get_info(self, var_names=None):
        out = {}
        for name, param in zip(self.model.param_names, self.result.params):
            out.append({name: pd.DataFrame({name: param})})
        return out

    def to_result(self):
        pass


In [ ]:
# only for testing model
from meteo_imp.kalman.random_walker import UnivarLocalLevel

In [ ]:
from datetime import datetime, date

In [ ]:
dates = pd.date_range(date.today(), periods=10)

In [ ]:
dates[4] = np.nan

TypeError: Index does not support mutable operations

In [ ]:
X = pd.DataFrame(np.arange(0, 10.)).set_index(pd.date_range(date.today(), periods=10))
X.iloc[4] = np.nan

X

,0
2022-11-13,0.0
2022-11-14,1.0
2022-11-15,2.0
2022-11-16,3.0
2022-11-17,NaN
2022-11-18,5.0
2022-11-19,6.0
2022-11-20,7.0
2022-11-21,8.0
2022-11-22,9.0


In [ ]:
X = X.drop(index=X.index[4])

In [ ]:
imp = StateSpaceImputation(X, UnivarLocalLevel, model_args = {'dates' : X.index, 'freq': 'D'}, int_index=False)

ValueError: The given frequency argument could not be matched to the given index.

In [ ]:
imp.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.91317D+00    |proj g|=  2.67378D-01

At iterate    5    f=  1.40283D+00    |proj g|=  5.38931D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      8     13      1     0     0   2.492D-06   1.403D+00
  F =   1.4028136878039665     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


<__main__.StateSpaceImputation>

In [ ]:
dates

DatetimeIndex(['2022-11-13', '2022-11-14', '2022-11-15', '2022-11-16',
               '2022-11-17', '2022-11-18', '2022-11-19', '2022-11-20',
               '2022-11-21', '2022-11-22'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
imp.result.predict()

2022-11-13    0.0
2022-11-14    0.0
2022-11-15    1.0
2022-11-16    2.0
2022-11-17    3.0
2022-11-18    5.0
2022-11-19    6.0
2022-11-20    7.0
2022-11-21    8.0
2022-11-22    9.0
Freq: D, Name: predicted_mean, dtype: float64

In [ ]:
imp.result.predict()

array([0., 0., 1., 2., 3., 5., 6., 7., 8.])

In [ ]:
p = imp.result.get_prediction(end=10)

In [ ]:
p.predicted_mean

array([0., 0., 1., 2., 3., 5., 6., 7., 8., 9., 9.])

In [ ]:
p.se_mean

array([1000.        ,    1.17260087,    1.17260087,    1.17260087,
          1.17260087,    1.17260087,    1.17260087,    1.17260087,
          1.17260087,    1.17260087,    1.65830805])

For testing purposes use models from statsmodel

In [ ]:
"""
Univariate Local Linear Trend Model
"""
class LocalLinearTrend(sm.tsa.statespace.MLEModel):
    def __init__(self, endog):
        # Model order
        k_states = k_posdef = 2

        # Initialize the statespace
        super(LocalLinearTrend, self).__init__(
            endog, k_states=k_states, k_posdef=k_posdef,
            initialization='approximate_diffuse',
            loglikelihood_burn=k_states
        )

        # Initialize the matrices
        self.ssm['design'] = np.array([[1, 0],
                                      [0, 0]])
        self.ssm['transition'] = np.array([[1, 1],
                                       [0, 1]])
        self.ssm['selection'] = np.eye(k_states)

        # Cache some indices
        self._state_cov_idx = ('state_cov',) + np.diag_indices(k_posdef)

    @property
    def param_names(self):
        return ['sigma2.measurement', 'sigma2.level', 'sigma2.trend']

    @property
    def start_params(self):
        return [np.std(self.endog)]*3

    def transform_params(self, unconstrained):
        return unconstrained**2

    def untransform_params(self, constrained):
        return constrained**0.5

    def update(self, params, *args, **kwargs):
        params = super(LocalLinearTrend, self).update(params, *args, **kwargs)

        # Observation covariance
        self.ssm['obs_cov',0,0] = params[0]

        # State covariance
        self.ssm[self._state_cov_idx] = params[1:]

In [ ]:
imp.model.fit()

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
np.array(1, ndmin=2)

array([[1]])

# Local level model
> simplest implementation of a kalman filter with a random walker or local level model

## Introduction

The model uses a Kalman filter with the simplest possible equations

The notation used is from @durbin_time_2012 since the one used in the statsmodel library that is used for this implementation.

The state of the model changes only due to the process noise (the design matrix is one)
The transition matrix is also one

The general notation is:

$$\begin{split}y_t & = Z_t \alpha_t + d_t + \varepsilon_t \\
\alpha_{t+1} & = T_t \alpha_t + c_t + R_t \eta_t \\\end{split}$$

where $$\begin{split}\varepsilon_t \sim N(0, H_t) \\
\eta_t \sim N(0, Q_t) \\\end{split}$$

- Z : `design`          $(k\_endog \times k\_states \times nobs)$
- d : `obs_intercept`   $k\_endog \times nobs)$
- H : `obs_cov`         $k\_endog \times k\_endog \times nobs)$
- T : `transition`      $k\_states \times k\_states \times nobs)$
- c : `state_intercept` $k\_states \times nobs)$
- R : `selection`       $k\_states \times k\_posdef \times nobs)$
- Q : `state_cov`       $k\_posdef \times k\_posdef \times nobs)$

### Local Level

The random walker makes the following assumptions:

- the design matrix is one, which means that the change over time of the state is only due to the random process noise
- the transition matrix is one, which means that the observations are equal to the state plus a random measurement noise
- the process and measurement noise don't change over time

Hence the equations of the model are:

$$\begin{split}
y_t & = \alpha_t +  \varepsilon \\
\alpha_{t+1} & = \alpha_t + \eta
\end{split}$$

The parameters of the models are $\varepsilon$ and $\eta$ which are estimated by maximising the log likelihood

this first implementation considers only a univariate scenario

## Implementation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import statsmodels.api as sm
from fastcore.basics import patch
import pandas as pd

In [ ]:
"""
Univariate Random Walker or local level model
"""
class UnivarLocalLevel(sm.tsa.statespace.MLEModel):
    def __init__(self, endog, **kwargs):
        # Model order
        k_states = k_posdef = 1

        # Initialize the statespace
        super(UnivarLocalLevel, self).__init__(
            endog, k_states=k_states, k_posdef=k_posdef,
            initialization='approximate_diffuse',
            loglikelihood_burn=k_states, **kwargs
        )

        # Initialize the matrices
        self.ssm['design'] = np.array([1])
        self.ssm['transition'] = np.array([1])
        self.ssm['selection'] = np.eye(k_states)

        # Cache some indices
        self._state_cov_idx = ('state_cov',) + np.diag_indices(k_posdef)

    @property
    def param_names(self):
        return ['epsilon', 'eta']

    @property
    def start_params(self):
        return [np.std(self.endog)]*2
    
    # This is to guarantee that the parameters are positive
    def transform_params(self, unconstrained):
        return unconstrained**2

    def untransform_params(self, constrained):
        return constrained**0.5

    def update(self, params, *args, **kwargs):
        params = super(UnivarLocalLevel, self).update(params, *args, **kwargs)

        # Observation covariance
        self.ssm['obs_cov',0,0] = params[0]

        # State covariance
        self.ssm['state_cov', 0, 0] = params[1]

In [ ]:
Y = np.arange(0,10)

In [ ]:
rw = UnivarLocalLevel(Y)

In [ ]:
res = rw.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.87798D+00    |proj g|=  2.84223D-01

At iterate    5    f=  1.27704D+00    |proj g|=  2.20201D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      7     13      1     0     0   1.860D-06   1.277D+00
  F =   1.2770446799271464     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


In [ ]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   10
Model:             UnivarRandomWalker   Log Likelihood                 -12.770
Date:                Sat, 12 Nov 2022   AIC                             29.541
Time:                        19:06:40   BIC                             29.935
Sample:                             0   HQIC                            28.690
                                 - 10                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
epsilon     1.454e-11      1.061   1.37e-11      1.000      -2.079       2.079
eta            1.0000   1.66e+05   6.03e-06      1.000   -3.25e+05    3.25e+05
===================================================================================
Ljung-Box (L1) (Q):                   0.38   Jarque-Bera (JB):                 2.70
Prob(Q):                              0.54   Prob(JB):                         0.26
Heteroskedasticity (H):               1.00   Skew:                            -1.34
Prob(H) (two-sided):                  1.00   Kurtosis:                         2.79
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
res.params

array([1.45422542e-11, 9.99987933e-01])

In [ ]:
predict = res.get_prediction()

In [ ]:
predict.se_mean

array([1.00000000e+03, 9.99993967e-01, 9.99993967e-01, 9.99993967e-01,
       9.99993967e-01, 9.99993967e-01, 9.99993967e-01, 9.99993967e-01,
       9.99993967e-01, 9.99993967e-01])

In [ ]:
res.get_prediction(1, 10)

<statsmodels.tsa.statespace.mlemodel.PredictionResultsWrapper>